In [55]:
import requests
import bs4 as bs
import lxml
httpaddr = "https://www.clarin.com"

requests_ret = requests.get(httpaddr)
print(requests_ret, end = ' || ')
soup = bs.BeautifulSoup(requests_ret.text, 'lxml')
# Notas:
# Everything semmantically a header, is tagged with <h2>. To all scrapping practical
# purposes, all headers except h2 are deemded irrelevant
articles = soup.find_all("article"); print("articles :", len(articles), end = ' / ')
sections = soup.find_all("section"); print("sections :", len(sections), end = ' / ')
h1s = soup.find_all("h1"); print("h1s :", len(h1s), end = ' / ')
h2s = soup.find_all("h2"); print("h2s :", len(h2s), end = ' / ')
h3s = soup.find_all("h3"); print("h3s :", len(h3s), end = ' / ')
h4s = soup.find_all("h4"); print("h4s :", len(h4s), end = ' / ')
h5s = soup.find_all("h5"); print("h5s :", len(h5s), end = ' / ')
h6s = soup.find_all("h6"); print("h6s :", len(h6s))
# only <h2>s hold relevance
# the only <h1> holds an SVG graphic

for discard_tag in ("script", "style"):
    for t in soup.find_all(discard_tag): t.extract()


<Response [200]> || articles : 113 / sections : 20 / h1s : 1 / h2s : 142 / h3s : 0 / h4s : 0 / h5s : 0 / h6s : 0


In [56]:
# Check for section nesting
# There's an almost strict hierarchical nesting in which articles are always inside some section
# The double belonging of to sections 2 and 3 are due to the fact that section 2 is nested inside section 3 (July 24, 2022)
print("+++ <section> NESTING CHECK :: ")
for (i, j) in [(i, j) for (i, section_i) in enumerate(sections) for (j, section_j) in enumerate(sections) if section_j in section_i.descendants]:
    print(("\t<section>[%" + str(len(str(len(sections)))) + "d] is nested in <section>[%" + str(len(str(len(sections)))) + "d]") % (j, i))

print("+++ <article> NESTING CHECK :: ")
for (i, j) in [(i, j) for (i, article_i) in enumerate(articles) for (j, article_j) in enumerate(articles) if article_j in article_i.descendants]:
    print(("\t<article>[%" + str(len(str(len(articles)))) + "d] is nested in <article>[%" + str(len(str(len(articles)))) + "d]") % (j, i))

# dangling <article>s
# these, the <article>s not-in-any-section, are the articles depicted as "LO MAS VISTO POR LOS SUSCRIPTORES" at the tail of the page
articles_not_in_sections = articles[:]              # copy articles ...
articles_in_sections_set = set()
for section in sections:
    for article in articles:
        if article in section.descendants:
            articles_in_sections_set = articles_in_sections_set | {article} # need to use this notation. set() notation consumes an interable and therefore iterates over article's members
for article in articles_in_sections_set:
    while article in articles_not_in_sections:      # while instead of if, just in case of repetition
        articles_not_in_sections.remove(article)    # ... then substract every article already under a <section>
print("+++ NUMBER OF <article>s NOT UNDER ANY <section> :", len(articles_not_in_sections))

# Report which <article> is nested in which <section>s
maxlen = max( (len('section[' + 'x' * len(str(len(sections))) + ']:' + str(section.attrs['class'])) for section in sections) )
for i, section in enumerate(sections):
    line = (('section[%' + str(len(str(len(sections)))) + 'd]: %s' + ' ' * 100) % (i, str(section.attrs['class']),))[:maxlen] + " ||"
    for article in articles: line += "X" if article in section.descendants else '.'; 
    line += "|"
    print(line)

+++ <section> NESTING CHECK :: 
	<section>[ 4] is nested in <section>[ 3]
+++ <article> NESTING CHECK :: 
+++ NUMBER OF <article>s NOT UNDER ANY <section> : 6
section[ 0]: ['container', 'no-p']                                         ||XXXXX............................................................................................................|
section[ 1]: ['container', 'no-p', 'listado']                              ||.....XXXX........................................................................................................|
section[ 2]: ['container', 'no-p', 'listado']                              ||.........XXXXXXXXXXXXXX..........................................................................................|
section[ 3]: ['container', 'listado', 'no-p']                              ||.......................XXXXXXXXXXXX..............................................................................|
section[ 4]: ['columnas']                                                

In [57]:
articles_by_section = [[] for _ in sections]    # ... beware of reference replication...
for i, section in enumerate(sections):
    for j, article in enumerate(articles):
        if article in section.descendants: 
            articles_by_section[i].append(article)

# for i, art_by_sect in enumerate(articles_by_section): print(("<section>[%0" + str(len(str(len(sections)))) + "d]") % (i, ), "contains", len(art_by_sect), "articles")

articles_by_section_dict = {i : l for (i, l) in enumerate(articles_by_section)}
articles_by_section_dict.update({'none' : articles_not_in_sections})
# for k in articles_by_section_dict:
#     print(("For articles in <section>[%" + str(max([len(str(k)) for k in articles_by_section_dict])) + "s] number of articles:") % k, len(articles_by_section_dict[k]))

idx_width = max([len(str(k)) for k in articles_by_section_dict])
val_width = max([len(str(len(articles_by_section_dict[k]))) for k in articles_by_section_dict])
for i, k in enumerate(articles_by_section_dict):
    print(("<section>[%" + str(idx_width) + "s] " + "= %" + str(val_width) + "s") % 
          (k, str(len(articles_by_section_dict[k]))), end = '\n' if (i + 1) % 6 == 0 else ' // ')


<section>[   0] =  5 // <section>[   1] =  4 // <section>[   2] = 14 // <section>[   3] = 12 // <section>[   4] =  6 // <section>[   5] =  7
<section>[   6] =  4 // <section>[   7] =  5 // <section>[   8] =  4 // <section>[   9] =  6 // <section>[  10] =  6 // <section>[  11] =  6
<section>[  12] =  6 // <section>[  13] =  6 // <section>[  14] =  3 // <section>[  15] =  1 // <section>[  16] =  6 // <section>[  17] =  4
<section>[  18] =  4 // <section>[  19] =  4 // <section>[none] =  6 // 

In [58]:
# Explore section hierarchy of attributes
for i, section in enumerate(sections):
    cursor = section
    for j in range(len(list(section.parents))):
        print('  ' * j, "<%s>[%d]" % (cursor.name, i), 'attrs ::', cursor.attrs)
        cursor = cursor.parent
    print('-' * 60)

 <section>[0] attrs :: {'data-name': 'cont1B', 'class': ['container', 'no-p'], 'data-classes': 'container ', 'data-type': 'container'}
   <div>[0] attrs :: {'class': ['container', 'father', 'no-p'], 'data-classes': 'container', 'data-type': 'container_father', 'data-name': 'container_father'}
     <div>[0] attrs :: {'class': ['content', 'home']}
       <div>[0] attrs :: {'class': ['wrapper', 'board-home', 'portada']}
         <body>[0] attrs :: {}
           <html>[0] attrs :: {'lang': 'es'}
------------------------------------------------------------
 <section>[1] attrs :: {'data-name': 'cont1C', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container no-p listado ', 'data-type': 'container'}
   <div>[1] attrs :: {'class': ['container', 'father', 'no-p'], 'data-classes': 'container', 'data-type': 'container_father', 'data-name': 'container_father'}
     <div>[1] attrs :: {'class': ['content', 'home']}
       <div>[1] attrs :: {'class': ['wrapper', 'board-home', 'portada'

In [59]:
# Explore section childrens
for i, section in enumerate(sections):
    print("<%s>[%d]" % (section.name, i), 'attrs ::', section.attrs)
    for j, ch in enumerate(section.children):
        print('    <%s>[%d]' % (ch.name if ch.name else type(ch), j), ('/ attrs:' + str(ch.attrs)) if ch.name else '' )
        # if ch.name:
        #     for k, gch in enumerate(ch):
        #         print('      <%s>[%d]' % (ch.name if ch.name else type(ch), k), ('/ attrs:' + str(ch.attrs)) if ch.name else '' )

<section>[0] attrs :: {'data-name': 'cont1B', 'class': ['container', 'no-p'], 'data-classes': 'container ', 'data-type': 'container'}
    <<class 'bs4.element.NavigableString'>>[0] 
    <ul>[1] / attrs:{'class': ['count-first', 'bb-container', 'one-floor', 'bb-moduleId_1B_mod_202261638535874'], 'data-count': '4', 'data-classes': ''}
    <<class 'bs4.element.NavigableString'>>[2] 
<section>[1] attrs :: {'data-name': 'cont1C', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container no-p listado ', 'data-type': 'container'}
    <<class 'bs4.element.NavigableString'>>[0] 
    <ul>[1] / attrs:{'class': ['count-first', 'bb-container', 'one-floor-C', 'bb-moduleId_1C_mod_202264672001627'], 'data-classes': ' ', 'data-count': '4'}
    <<class 'bs4.element.NavigableString'>>[2] 
<section>[2] attrs :: {'data-name': 'cont4B', 'class': ['container', 'no-p', 'listado'], 'data-classes': 'container listado ', 'data-type': 'container'}
    <<class 'bs4.element.NavigableString'>>[0] 
    <u

In [60]:
# dumping some data by article as classified in articles_by_section_dict
for k, a_list in articles_by_section_dict.items():
    for j, article in enumerate(a_list):
        first_h2 = article.find('h2').get_text() if article.find('h2') else None
        h2ss = article.find_all('h2') if article.find_all('h2') else None
        volanta = article.find(class_ = 'volanta').get_text() if article.find(class_ = 'volanta') else None
        author = article.find(class_ = 'author').get_text() if article.find(class_ = 'author') else None
        summary = article.find(class_ = 'summary').get_text() if article.find(class_ = 'summary') else None
        print("<section>[%s] / <article>[%d]" % (str(k), j))
        for i, h in enumerate(h2ss):
            print('%s\th2[%d] = %s' % ('\t' if i != 0 else '', i, h))
        print( '\tvolanta = "%s"' % (volanta, ),
               '\n\tauthor = "%s"' % (author, ),
               '\n\tsummary = "%s"' % (summary, ))
    print('-' * 60)

<section>[0] / <article>[0]
	h2[0] = <h2>Tras los cambios en el Gobierno, el dólar blue acelera su descenso y perfora el piso de los $ 300</h2>
	volanta = "Reacción de los mercados" 
	author = "None" 
	summary = "También bajan fuerte el contado con liqui y el MEP. Los bonos rebotan más del 5% y el riesgo país retrocede a casi 2.400, el nivel previo a la renuncia de Guzmán como ministro."
<section>[0] / <article>[1]
	h2[0] = <h2 class="title-rel">Antes de llegar a Economía, Massa visitó a Lavagna: lo tendrá como consultor</h2>
	volanta = "None" 
	author = "None" 
	summary = "None"
<section>[0] / <article>[2]
	h2[0] = <h2>Massa dijo que el miércoles anunciará medidas y planteó: “No soy ningún salvador”</h2>
	volanta = "El nuevo superministro" 
	author = "None" 
	summary = "None"
<section>[0] / <article>[3]
	h2[0] = <h2>Las primeras definiciones de Alberto Fernández sobre el nuevo rol de Massa</h2>
	volanta = "Se reunieron hoy en Olivos" 
	author = "None" 
	summary = "None"
<section>[0] /

In [61]:
for i, article in enumerate(articles_by_section[0]):
    print("<article>[%d]" % (i, ))
    print("-------------")
    print(len(article.parent.find_all('li', class_ = 'item-related')))
    print(article.parent.prettify())
    print('-' * 60)

<article>[0]
-------------
1
<li class="m-box child h300 col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p m2x1" data-classes="m-box child m2x1 h300  col-lg-6 col-md-8 col-sm-12 col-xs-12 no-p" data-name="content" data-size-h="1" data-size-w="2">
 <article class="content-nota list-format twoxone_no_foto">
  <div class="mt">
   <div class="max-h flex-change">
    <h2>
     Tras los cambios en el Gobierno, el dólar blue acelera su descenso y perfora el piso de los $ 300
    </h2>
    <p class="volanta">
     Reacción de los mercados
    </p>
   </div>
   <div class="data-txt">
    <p class="section">
     Economía
    </p>
   </div>
   <p class="summary">
    También bajan fuerte el contado con liqui y el MEP. Los bonos rebotan más del 5% y el riesgo país retrocede a casi 2.400, el nivel previo a la renuncia de Guzmán como ministro.
   </p>
  </div>
  <a aria-label="/economia/dolar-blue-dolar-hoy-vivo-abre-cotizacion-precio-29-julio-minuto-minuto_0_sG6OGfvBGj.html" class="link_article" href="/e

In [62]:
# How many <li>s are on each <article>?

for i, article in enumerate(articles):
    print("<article>[%3s]" % i, str(len(list(article.parent.find_all('li')))) if len(list(article.parent.find_all('li'))) != 0 else "", 
          end = ' ' if (i+1) % 8 != 0 else '\n')

<article>[  0] 1 <article>[  1]  <article>[  2]  <article>[  3]  <article>[  4]  <article>[  5]  <article>[  6]  <article>[  7] 
<article>[  8]  <article>[  9]  <article>[ 10]  <article>[ 11]  <article>[ 12]  <article>[ 13]  <article>[ 14]  <article>[ 15] 
<article>[ 16]  <article>[ 17]  <article>[ 18]  <article>[ 19]  <article>[ 20]  <article>[ 21]  <article>[ 22]  <article>[ 23] 
<article>[ 24]  <article>[ 25]  <article>[ 26]  <article>[ 27]  <article>[ 28]  <article>[ 29]  <article>[ 30]  <article>[ 31] 
<article>[ 32]  <article>[ 33]  <article>[ 34]  <article>[ 35]  <article>[ 36]  <article>[ 37]  <article>[ 38]  <article>[ 39] 
<article>[ 40]  <article>[ 41]  <article>[ 42]  <article>[ 43]  <article>[ 44]  <article>[ 45]  <article>[ 46]  <article>[ 47] 
<article>[ 48]  <article>[ 49]  <article>[ 50]  <article>[ 51]  <article>[ 52]  <article>[ 53]  <article>[ 54]  <article>[ 55] 
<article>[ 56] 11 <article>[ 57]  <article>[ 58]  <article>[ 59]  <article>[ 60]  <article>[ 61]  <arti

In [63]:
for i, art in enumerate(articles_by_section_dict[1][7:]):
    print(i, '\n', art.parent.parent.parent)

In [64]:
divs = soup.find('body').find_all('div', class_ = 'container father no-p')
# print(len(divs))
def dumptree(n, prefix):
    global articles
    global sections
    stack = []
    stack.extend(reversed([ch for ch in n.children if type(ch) == bs.element.Tag]))     # Warning! Generators tend to explode...
    print(prefix + n.name + ' // ', end = '')
    try:
        if n.name == 'h2' : print(n.attrs, '/', n.get_text().strip())
        elif n.name == 'p' : print(n.attrs, '/', n.get_text().strip())
        elif n.name == 'span' : print(n.attrs, '/', n.get_text().strip())
        elif  n.name == 'a' : print(n.attrs['href'], '/', ' '.join(n.get_text().strip().split('\n')))
        elif  n.name == 'div' : print(n.attrs['class'])
        elif  n.name == 'section':
            print(n.name + '[' + str(sections.index(n)) + ']')
        elif  n.name == 'article':
            print(n.name + '[' + str(articles.index(n)) + ']')
        else: print()
    except KeyError:
        print("+++ ERROR WHILE TRYING TO ACCESS NONEXISTANT KEY!!!+++")
    while stack:
        dumptree(stack.pop(), '| ' + prefix)
    return None


i = 0
print("<section>[%02d]" % i)
print("-------------")
dumptree(sections[i].parent, '')


<section>[00]
-------------
div // ['container', 'father', 'no-p']
| section // section[0]
| | ul // 
| | | li // 
| | | | article // article[0]
| | | | | div // ['mt']
| | | | | | div // ['max-h', 'flex-change']
| | | | | | | h2 // {} / Tras los cambios en el Gobierno, el dólar blue acelera su descenso y perfora el piso de los $ 300
| | | | | | | p // {'class': ['volanta']} / Reacción de los mercados
| | | | | | div // ['data-txt']
| | | | | | | p // {'class': ['section']} / Economía
| | | | | | p // {'class': ['summary']} / También bajan fuerte el contado con liqui y el MEP. Los bonos rebotan más del 5% y el riesgo país retrocede a casi 2.400, el nivel previo a la renuncia de Guzmán como ministro.
| | | | | a // /economia/dolar-blue-dolar-hoy-vivo-abre-cotizacion-precio-29-julio-minuto-minuto_0_sG6OGfvBGj.html / 
| | | | ul // 
| | | | | li // 
| | | | | | article // article[1]
| | | | | | | a // /politica/sergio-massa-visito-roberto-lavagna-pedirle-bendicion-ingreso-ministerio-econo

In [108]:
for i, section in enumerate(sections):
    if section.parent.header:
        print("<section>[%d]" % i, "has a header under it")
        for j, s in enumerate(section.parent):
            print(s.name)
            if j >= 2 : break

<section>[3] has a header under it
None
section
None
<section>[6] has a header under it
None
header
None
<section>[7] has a header under it
None
header
None
<section>[8] has a header under it
None
header
None
<section>[9] has a header under it
None
div
None
<section>[10] has a header under it
None
header
None
<section>[11] has a header under it
None
header
None
<section>[12] has a header under it
None
header
None
<section>[13] has a header under it
None
header
None
<section>[14] has a header under it
None
header
None
<section>[15] has a header under it
None
div
None
<section>[16] has a header under it
None
header
None
<section>[17] has a header under it
None
header
None
<section>[18] has a header under it
None
header
None
<section>[19] has a header under it
None
header
None
